<a href="https://colab.research.google.com/github/heber-augusto/sus-kpis-analysis/blob/main/sia/indicadores_monitor_rosa_sia_pa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apresentação
Este notebook foi desenolvido como parte do projeto Monitor Rosa, um conjunto de ferramentas, softwares e outros artefatos cujo principal objetivo é melhorar o cenário de diagnósticos de câncer de mama no Brasil, idealmente melhorando diminuindo a proporção de diagnósticos tardios/diagnósticos precoces.

Os arquivos utilizados neste notebook foram coletados através dos scripts presentes [neste repositório](https://github.com/heber-augusto/devops-pysus-get-files). Os arquios foram armazenados em um bucket do Google Storage no formato parquet compactados (gzip).

# Instalação de bibliotecas e pacotes para leitura de arquivos

## Configurações iniciais para conectar com bucket no Google Storage
 - Autenticação do Google Colab
 - Definição do nome do projeto

 Para a execução dos comandos desta seção, o arquio gcp-leitura.json deve ser inserido na raiz do colab.

In [2]:
from google.colab import auth
auth.authenticate_user()

# id do projeto
project_id = 'teak-ellipse-317015'
# id do bucket dentro do projeto
bucket_id = 'observatorio-oncologia'

# nome da pasta local para mapear
local_folder_name = 'colab'

# nome da pasta do projeto
project_folder_name = 'monitor'

!gcloud config set project {project_id}

Updated property [core/project].


## Instalação para garantir montagem da pasta no bucket
Instalação do gcsfuse para mapear pasta do bucket no google colab

In [3]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  84566      0 --:--:-- --:--:-- --:--:-- 84566
OK
73 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 73 not upgraded.
Need to get 11.6 MB of archives.
After this operation, 27.4 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 155653 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.41.4_amd64.deb ...
Unpacking gcsfuse (0.41.4) ...
Setting up gcsfuse (0.41.4) ...


## Nome/caminho do arquivo json contendo credenciais de acesso ao storage do Google

In [4]:
serice_account_json = '/content/gcp-leitura.json'

## Montagem do bucket em uma pasta local do google colab

In [5]:
!mkdir {local_folder_name}
!gcsfuse --key-file {serice_account_json} --implicit-dirs {bucket_id} {local_folder_name}

2022/07/29 11:11:45.956840 Start gcsfuse/0.41.4 (Go version go1.17.6) for app "" using mount point: /content/colab
2022/07/29 11:11:45.971617 Opening GCS connection...
2022/07/29 11:11:46.363059 Mounting file system "observatorio-oncologia"...
2022/07/29 11:11:46.398408 File system has been successfully mounted.


## Função para facilitar coleta de arquivo

In [6]:
import glob
import os

def get_files(state, year, month, file_type, file_group):
    initial_path = os.path.join(r'/content/',local_folder_name,project_folder_name)
    internal_folder = f"""{state}/{year}/{month}/{file_type}/{file_group}"""
    # print(f"{initial_path}/{internal_folder}/*.parquet.gzip")
    return glob.glob(f"{initial_path}/{internal_folder}/*.parquet.gzip")    

# SIA PA, AQ e AR: Leitura, filtro e transformação inicial dos arquivos

## Informações sobre filtros pertinentes ao contexto de câncer de mama:

### SIH: realizar filtro através da variável DIAG_PRINC (4 caracteres)
 * Filtro: C500, C501, C502, C503, C504, C505, C506, C508 e C509

### RHC: realizar filtro através da variável LOCTUDET (3 caracteres)
 * Filtro: C50

### SIA – APAC de Quimioterapia e Radioterapia (AQ e AR)
Realizar filtro através da variável AP_CIDPRI (4 caracteres)
 * Filtro: C500, C501, C502, C503, C504, C505, C506, C508 e C509

### SIA – Procedimentos ambulatoriais (PA)

Os arquivos de procedimentos ambulatoriais são um pouco diferentes por um motivo: a pessoa já pode ter o diagnóstico e está realizando um procedimento OU a pessoa está realizando um exame com finalidade diagnóstica (mamografia, ultrassonografia, etc). Então, neste caso, podemos pensar em dois filtros:

Realizar filtro através da variável PA_CIDPRI (4 caracteres)
 * Filtro: C500, C501, C502, C503, C504, C505, C506, C508 e C509

Realizar filtro através da variável do código de procedimento ambulatorial “PA_PROC_ID” (10 caracteres)
 * Filtros:
  * 201010569	BIOPSIA/EXERESE DE NÓDULO DE MAMA
  * 201010585	PUNÇÃO ASPIRATIVA DE MAMA POR AGULHA FINA
  * 201010607	PUNÇÃO DE MAMA POR AGULHA GROSSA
  * 203010035	EXAME DE CITOLOGIA (EXCETO CERVICO-VAGINAL E DE MAMA)
  * 203010043	EXAME CITOPATOLOGICO DE MAMA
  * 203020065	EXAME ANATOMOPATOLOGICO DE MAMA - BIOPSIA
  * 203020073	EXAME ANATOMOPATOLOGICO DE MAMA - PECA CIRURGICA
  * 205020097	ULTRASSONOGRAFIA MAMARIA BILATERAL
  * 208090037	CINTILOGRAFIA DE MAMA (BILATERAL)
  * 204030030	MAMOGRAFIA
  * 204030188	MAMOGRAFIA BILATERAL PARA RASTREAMENTO

## Bibliotecas utilizadas na análise exploratória

In [7]:
import pandas as pd
import numpy as np 

## Variáveis de filtro

In [8]:
# filtro pelo cid
cid_filter = ['C500', 'C501', 'C502', 'C503', 'C504', 'C505', 'C506', 'C508', 'C509']

# dicionario de procedimentos
proc_id_dict = {
    '0201010569': 'BIOPSIA/EXERESE DE NÓDULO DE MAMA',
    '0201010585': 'PUNÇÃO ASPIRATIVA DE MAMA POR AGULHA FINA',
    '0201010607': 'PUNÇÃO DE MAMA POR AGULHA GROSSA',
    '0203010035': 'EXAME DE CITOLOGIA (EXCETO CERVICO-VAGINAL E DE MAMA)',
    '0203010043': 'EXAME CITOPATOLOGICO DE MAMA',
    '0203020065': 'EXAME ANATOMOPATOLOGICO DE MAMA - BIOPSIA',
    '0203020073': 'EXAME ANATOMOPATOLOGICO DE MAMA - PECA CIRURGICA',
    '0205020097': 'ULTRASSONOGRAFIA MAMARIA BILATERAL',
    '0208090037': 'CINTILOGRAFIA DE MAMA (BILATERAL)',
    '0204030030': 'MAMOGRAFIA',
    '0204030188': 'MAMOGRAFIA BILATERAL PARA RASTREAMENTO'
    }
proc_id_filter = list(proc_id_dict.keys())


## Funções de filtro para arquivo SIA PA, AQ e AR

In [9]:
def filter_pa_content(df):
    """

    """
    return df[df.PA_CIDPRI.isin(cid_filter) & \
              df.PA_PROC_ID.isin(proc_id_filter)]

def filter_ar_content(df):
    """

    """
    return df[df.AP_CIDPRI.isin(cid_filter)]

filter_aq_content = filter_ar_content


## Função para unir diversos arquivos em um único datraframe

In [10]:
def create_cancer_dataframe(file_paths, filter_function=filter_pa_content):
    """

    """
    filtered_contents = [
      filter_function(pd.read_parquet(file_path))
      for file_path in file_paths
      ]

    return pd.concat(
        filtered_contents, 
        ignore_index=True)



## Função para retornar lista de arquivos (caminho completo)

In [11]:
def get_file_paths(states, years, months, file_type, file_group):
    """

    """
    file_paths = []
    for state in states:
        for year in years:
            for month in months:
                file_paths.extend(
                    get_files(
                        state,
                        year,
                        month,
                        file_type,
                        file_group)
                )
    return file_paths

## SIA PA: Leitura e união de dados para o período desejado




### Estado, anos e meses a serem lidos e processados

In [12]:
states = ['SP',]
years  = ['2020','2021']
months = [f'{month + 1:02d}' for month in range(12)]
file_type = 'SIA'

### Monta lista de arquivos a serem lidos

In [13]:
file_paths_by_type = {}

# Arquivos de produção ambulatorial
file_paths_by_type['PA'] = get_file_paths(
    states,
    years,
    months,
    file_type,
    'PA'
)

# Arquivos de radioterapia
file_paths_by_type['AR'] = get_file_paths(
    states,
    years,
    months,
    file_type,
    'AR'
)

# Arquivos de quimioteraia
file_paths_by_type['AQ'] = get_file_paths(
    states,
    years,
    months,
    file_type,
    'AQ'
)

In [14]:
print(f"""Identificados {len(file_paths_by_type['PA'])} arquivos de produção ambulatorial""")
print(f"""Identificados {len(file_paths_by_type['AR'])} arquivos de radioterapia""")
print(f"""Identificados {len(file_paths_by_type['AQ'])} arquivos de quimioterapia""")

Identificados 72 arquivos de produção ambulatorial
Identificados 24 arquivos de radioterapia
Identificados 24 arquivos de radioterapia


## Cria um único dataframe a partir dos conteúdos filtrados

In [ ]:
cancer_dataframe_pa = create_cancer_dataframe(file_paths_by_type['PA'], filter_function=filter_pa_content)
cancer_dataframe_aq = create_cancer_dataframe(file_paths_by_type['AQ'], filter_function=filter_ar_content)
cancer_dataframe_ar = create_cancer_dataframe(file_paths_by_type['AR'], filter_function=filter_aq_content)

## Dataframe de PA

In [ ]:
cancer_dataframe_pa

In [ ]:
cancer_dataframe_pa.info()

## Dataframe de AQ

In [ ]:
cancer_dataframe_aq

In [ ]:
cancer_dataframe_aq.info()

## Dataframe de AR

In [ ]:
cancer_dataframe_ar

In [ ]:
cancer_dataframe_ar.info()

# Montagem do dataset de Exames de Paciente (1 linha por paciente)

Colunas:

 - Chave da paciente (cns_encrypted)
 - Custo total do tratamento
 - Estadiamento inicial
 - Estadiamento final
 - Indicação de óbito
 - Localização de moradia

## Proposta de solução:

 - Tipos de arquivos a serem utilizados: AQ e AR
 - chave do paciente:
  - AQ: coluna AP_CNSPCN
  - AR: coluna AP_CNSPCN  
 - custo total do tratamento: será uma estimativa considerando apenas radioterapia e quimioterapia, calculado somando os valores dos procedimentos (presentes em AR e AQ), para cada chave de paciente. Sendo:
   - custos em AQ: soma de AP_VL_AP;
   - custos em AR: soma de AP_VL_AP.
 - Estadiamento inicial: calculado utilizando o valor do estadiamento (presentes em AQ:AQ_ESTADI e AR:AR_ESTADI) do registro mais antigo de radioterapia ou quimioterapia, de um determinado paciente;
 - Estadiamento final: calculado utilizando o valor do estadiamento (presentes em AQ:AQ_ESTADI e AR:AR_ESTADI) do registro mais recente de radioterapia ou quimioterapia, de um determinado paciente;
 - Localização de moradia: utilizar coluna AP_MUNPCN (presentes em AR e AQ). Talvez criar dois campos, AP_MUNPCN presente no registro mais antigo e AP_MUNPCN presente no registro mais novo